In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models

c:\Users\nikhi\anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [ ]:
data = pd.read_csv('/content/student_data.csv')
data = pd.get_dummies(data)

In [ ]:
num_features = len(data.columns)
latent_dim = 100  # Example dimension of latent space
num_epochs = 100
batch_size = 32

In [ ]:
def build_generator(latent_dim):
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu', input_dim=latent_dim))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dense(num_features, activation='sigmoid'))
    return model

In [ ]:
def build_discriminator(num_features):
    model = models.Sequential()
    model.add(layers.Dense(256, activation='relu', input_dim=num_features))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    return model

In [ ]:
discriminator = build_discriminator(num_features)
discriminator.compile(loss='binary_crossentropy', optimizer='adam')

In [ ]:
generator = build_generator(latent_dim)
gan_input = tf.keras.Input(shape=(latent_dim,))
fake_student = generator(gan_input)
gan_output = discriminator(fake_student)
gan = tf.keras.models.Model(gan_input, gan_output)
discriminator.trainable = False
gan.compile(loss='binary_crossentropy', optimizer='adam')

In [ ]:
X_train = data.values

In [ ]:
discriminator_losses = []
for epoch in range(num_epochs):
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    fake_student_data = generator.predict(noise)
    idx = np.random.randint(0, X_train.shape[0], batch_size)
    real_student_data = X_train[idx]
    d_loss_real = discriminator.train_on_batch(real_student_data, np.ones((batch_size, 1)))
    d_loss_fake = discriminator.train_on_batch(fake_student_data, np.zeros((batch_size, 1)))
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
    discriminator_losses.append(d_loss)
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    g_loss = gan.train_on_batch(noise, np.ones((batch_size, 1)))
    print(f"Epoch: {epoch+1}, Discriminator Loss: {d_loss}, Generator Loss: {g_loss}")

threshold = 0.1

In [ ]:
fake_students = []
real_students = []

for epoch, discriminator_loss in enumerate(discriminator_losses, 1):
    if discriminator_loss < threshold:
        fake_students.append((epoch, discriminator_loss))
    else:
        real_students.append((epoch, discriminator_loss))

In [ ]:
print("Fake Students:")
for student in fake_students:
    print("Epoch:", student[0], "Discriminator Loss:", student[1])

In [ ]:
print("\nReal Students:")
for student in real_students:
    print("Epoch:", student[0], "Discriminator Loss:", student[1])